Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All).

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE", as well as your name and collaborators below:

In [ ]:
NAME = ""
COLLABORATORS = ""

---

<!--NOTEBOOK_HEADER-->
*This notebook contains material from [PyRosetta](https://RosettaCommons.github.io/PyRosetta.notebooks);
content is available [on Github](https://github.com/RosettaCommons/PyRosetta.notebooks.git).*

<!--NAVIGATION-->
< [Packing and Relax](http://nbviewer.jupyter.org/github/RosettaCommons/PyRosetta.notebooks/blob/master/notebooks/06.02-Packing-design-and-regional-relax.ipynb) | [Contents](toc.ipynb) | [Index](index.ipynb) | [Protein Design 2](http://nbviewer.jupyter.org/github/RosettaCommons/PyRosetta.notebooks/blob/master/notebooks/06.04-Protein-Design-2.ipynb) ><p><a href="https://colab.research.google.com/github/RosettaCommons/PyRosetta.notebooks/blob/master/notebooks/06.03-Design-with-a-resfile-and-relax.ipynb"><img align="left" src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab" title="Open in Google Colaboratory"></a>

# Protein Design with a Resfile and FastRelax

Keywords: FastDesign, FastRelax, ResfileCommandOperation, Resfile, ResidueSelector, MoveMapFactory, TaskFactory, TaskOperation, NoRepackDisulfides, IncludeCurrent, ReadResfile, conf2pdb_chain(), pose_from_rcsb(), create_score_function(), CA_rmsd()

## Overview

In this Workshop, we will learn the classic way to design proteins, but in the same breath introduce the concept of design using a flexible backbone protocol.  

This protocol, is essentially design during FastRelax.  A separate class, FastDesign, has a bit more options for design, but essentially, they are the same.  

Many modern designs have used this FastDesign/RelaxedDesign protocol - including many Science papers from the Baker lab and the RosettaAntibodyDesign (RAbD) protocol that we will cover in another tutorial. 

Before this workshop, you should read about the resfile syntax here: https://www.rosettacommons.org/docs/latest/rosetta_basics/file_types/resfiles

*Warning*: This notebook uses `pyrosetta.distributed.viewer` code, which runs in `jupyter notebook` and might not run if you're using `jupyterlab`.

In [ ]:
# Notebook setup
import sys
if 'google.colab' in sys.modules:
    !pip install pyrosettacolabsetup
    import pyrosettacolabsetup
    pyrosettacolabsetup.setup()
    print ("Notebook is set for PyRosetta use in Colab.  Have fun!")

**Make sure you are in the directory with the pdb files:**

`cd google_drive/My\ Drive/student-notebooks/`

In [ ]:
import logging
logging.basicConfig(level=logging.INFO)
import pyrosetta
import pyrosetta.toolbox

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

### Initialize PyRosetta

In [ ]:
pyrosetta.init("-ignore_unrecognized_res 1 -ex1 -ex2aro -detect_disulf 0")

 For this tutorial, let's use the well-studied native protein crambin from PDB ID 1AB1 (http://www.rcsb.org/structure/1AB1).
 
 Setup the input pose and scorefunction:

In [ ]:
start_pose = pyrosetta.toolbox.rcsb.pose_from_rcsb("1AB1", ATOM=True, CRYS=False)
pose = start_pose.clone()
scorefxn = pyrosetta.create_score_function("ref2015_cart.wts")

 Make of list of which residues are cysteine:

In [ ]:
cys_res = []
for i, aa in enumerate(start_pose.sequence(), start=1):
    if aa == "C":
        cys_res.append(i)
print(cys_res)

Inspect `start_pose` using the `PyMolMover` or `dump_pdb()`

## Design strategy:

Design away the cysteine residues (i.e. disulfide bonds) using a resfile, allowing all side-chains to re-pack and all backbone and side-chain torsions to minimize using the `FastRelax` mover.

 Read more about resfile file structure at https://www.rosettacommons.org/docs/latest/rosetta_basics/file_types/resfiles

To write a resfile, we need to know which chain to mutate.

We can see that the pose consists of only chain "A" by printing the `pose.pdb_info()` object:

In [ ]:
print(pose.pdb_info())

 More programmatically, we could find which chains are in the `pose` using `pyrosetta.rosetta.core.pose.conf2pdb_chain(pose)` which returns a `pyrosetta.rosetta.std.map_unsigned_long_char` object which is iterable.

In [ ]:
print(pyrosetta.rosetta.core.pose.conf2pdb_chain(pose))

In [ ]:
for k, v in pyrosetta.rosetta.core.pose.conf2pdb_chain(pose).items():
    print(v)

 So we could write a resfile to disc indicating design specifications to mutate only the cysteine residues in chain "A".  Use the syntax described below, and save your resfile in this directory as `resfile`.
 
 https://www.rosettacommons.org/docs/latest/rosetta_basics/file_types/resfiles

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

Note that we don't necessarily need a resfile to use resfile commands.  We can now do this in an intuitive way through code and `ResidueSelectors` using the `ResfileCommandOperation`.  The main docs for the XML interface are available below, however the code-level interface is extremely similar.  Use the ? to get more info on this.  The operation is located in `pyrosetta.rosetta.core.pack.task.operation` as we saw this location in the previous tutorial.

https://www.rosettacommons.org/docs/latest/scripting_documentation/RosettaScripts/TaskOperations/taskoperations_pages/ResfileCommandOperation

 Now we can setup the TaskOperations for the `FastRelax` mover. These tell `FastRelax` which residues to design or repack during the packer steps in `FastRelax`.  You should be familiar with this from the previous tutorial
 
We use the `IncludeCurrent` to include the current rotamer of from the crystal structure during packing.

In [ ]:
# The task factory accepts all the task operations
tf = pyrosetta.rosetta.core.pack.task.TaskFactory()

# These are pretty standard
tf.push_back(pyrosetta.rosetta.core.pack.task.operation.InitializeFromCommandline())
tf.push_back(pyrosetta.rosetta.core.pack.task.operation.IncludeCurrent())
tf.push_back(pyrosetta.rosetta.core.pack.task.operation.NoRepackDisulfides())

# Include the resfile
tf.push_back(pyrosetta.rosetta.core.pack.task.operation.ReadResfile(resfile))

# Convert the task factory into a PackerTask to take a look at it
packer_task = tf.create_task_and_apply_taskoperations(pose)
# View the PackerTask
print(packer_task)

 The PackerTask looks as intended! 

 Now we can set up a `MoveMap` or a `MoveMapFactory` to specify which torsions are free to minimize during the minimization steps of the `FastDesign` mover

In [ ]:
# Set up a MoveMapFactory
mmf = pyrosetta.rosetta.core.select.movemap.MoveMapFactory()
mmf.all_bb(setting=True)
mmf.all_bondangles(setting=True)
mmf.all_bondlengths(setting=True)
mmf.all_chi(setting=True)
mmf.all_jumps(setting=True)
mmf.set_cartesian(setting=True)

In [ ]:
# Set up a MoveMap
# mm = pyrosetta.rosetta.core.kinematics.MoveMap()
# mm.set_bb(True)
# mm.set_chi(True)
# mm.set_jump(True)

# If needed, you could turn off bb and chi torsions for individual residues like this:

# vector1 of true/false for each residue in the pose
# subset_to_minimize = do_something_set.apply(pose)

# for i in range(1, pose.size() + 1):
#     if (not subset_to_minimize[i]):
#         mm.set_bb(i, False)
#         mm.set_chi(i, False)

 Because some Movers only take as input a `MoveMap`, for backwards-compatibility one could generate a `MoveMap` from a `MoveMapFactory` using the `MoveMapFactory` function `create_movemap_from_pose(pose)`

 Now let's double-check some more `pose` information to verify that we are ready for `FastRelax`:

In [ ]:
display_pose = pyrosetta.rosetta.protocols.fold_from_loops.movers.DisplayPoseLabelsMover()
display_pose.tasks(tf)
display_pose.movemap_factory(mmf)
display_pose.apply(pose)

 Setting up `FastRelax` prints the default `relaxscript`, showing the `ramp_repack_min` settings with the following assignments:
>ramp_repack_min [scale:fa_rep] [min_tolerance] [coord_cst_weight]

In [ ]:
fr = pyrosetta.rosetta.protocols.relax.FastRelax(scorefxn_in=scorefxn, standard_repeats=1)
fr.cartesian(True)
fr.set_task_factory(tf)
fr.set_movemap_factory(mmf)
fr.min_type("lbfgs_armijo_nonmonotone") # For non-Cartesian scorefunctions, use "dfpmin_armijo_nonmonotone"

#Note that this min_type is automatically set when you set the cartesian option. 
#  But it is good to be aware of this - as not all protocols will do this for you.

#fr.set_movemap(mm) # Could have optionally specified a MoveMap instead of MoveMapFactory
#fr.minimize_bond_angles(True) # If not using MoveMapFactory, could specify bond angle minimization here
#fr.minimize_bond_lengths(True) # If not using MoveMapFactory, could specify bond length minimization here

For recommendations on setting `fr.min_type()` for the scorefunction being used, see: https://www.rosettacommons.org/docs/latest/rosetta_basics/structural_concepts/minimization-overview#recommendations

 Run Fast(Design)! Note: this takes ~1min 31s

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

### Analysis

Inspect the resulting design!

 By how many Angstroms RMSD did the backbone Cα atoms move?

In [ ]:
pyrosetta.rosetta.core.scoring.CA_rmsd(start_pose, pose)

What is the delta `total_score` from `start_pose` to `pose`? Why is it large?

In [ ]:
delta_total_score = scorefxn(pose) - scorefxn(start_pose)
print(delta_total_score)

 What is the per-residue energy difference for each mutated position between `start_pose` and `pose`?

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

<!--NAVIGATION-->
< [Packing and Relax](http://nbviewer.jupyter.org/github/RosettaCommons/PyRosetta.notebooks/blob/master/notebooks/06.02-Packing-design-and-regional-relax.ipynb) | [Contents](toc.ipynb) | [Index](index.ipynb) | [Protein Design 2](http://nbviewer.jupyter.org/github/RosettaCommons/PyRosetta.notebooks/blob/master/notebooks/06.04-Protein-Design-2.ipynb) ><p><a href="https://colab.research.google.com/github/RosettaCommons/PyRosetta.notebooks/blob/master/notebooks/06.03-Design-with-a-resfile-and-relax.ipynb"><img align="left" src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab" title="Open in Google Colaboratory"></a>